In [ ]:
import os
from dotenv import load_dotenv
import huggingface_hub

# load env var huggingface token
load_dotenv(os.path.join("../", ".env"))
# login to the hub
huggingface_hub.login(token=os.getenv("HUGGINGFACE_TOKEN"))

In [29]:
import datasets

voxpopuli = datasets.load_dataset(
    "facebook/voxpopuli", "en", streaming=True, trust_remote_code=True
)

In [30]:
voxpopuli_head = list(voxpopuli["train"].take(5))
SAMPLING_RATE = voxpopuli_head[0]["audio"]["sampling_rate"]
print(voxpopuli_head)

[{'audio_id': '20180418-0900-PLENARY-3-en_20180418-08:50:36_17', 'language': 0, 'audio': {'path': 'train_part_0/20180418-0900-PLENARY-3-en_20180418-08:50:36_17.wav', 'array': array([-0.00030518,  0.00119019,  0.00506592, ..., -0.00036621,
       -0.00027466, -0.00018311]), 'sampling_rate': 16000}, 'raw_text': 'If you do not address this problem, the ground is there for populist nationalist forces to go on growing all over Europe.', 'normalized_text': 'if you do not address this problem the ground is there for populist nationalist forces to go on growing all over europe.', 'gender': 'female', 'speaker_id': '124737', 'is_gold_transcript': True, 'accent': 'None'}, {'audio_id': '20170614-0900-PLENARY-5-en_20170614-10:03:08_5', 'language': 0, 'audio': {'path': 'train_part_0/20170614-0900-PLENARY-5-en_20170614-10:03:08_5.wav', 'array': array([-0.00036621, -0.00030518, -0.00042725, ...,  0.00012207,
        0.00119019,  0.00027466]), 'sampling_rate': 16000}, 'raw_text': 'they attacked and rem

In [ ]:
from transformers import (
    AutoTokenizer,
    AutoFeatureExtractor,
    SpeechEncoderDecoderModel,
    SpeechEncoderDecoderConfig,
    AutoConfig,
)

import torch

encoder_id = "facebook/wav2vec2-base-960h"  # acoustic model encoder
decoder_id = "facebook/bart-base"  # text decoder

# feature_extractor = AutoFeatureExtractor.from_pretrained(encoder_id)
# tokenizer = AutoTokenizer.from_pretrained(decoder_id)

model = SpeechEncoderDecoderModel.from_encoder_decoder_pretrained(
    encoder_id, decoder_id, encoder_add_adapter=True
)
model.config.encoder.feat_proj_dropout = 0.0
model.config.encoder.mask_time_prob = 0.0
model.config.decoder_start_token_id = model.decoder.config.bos_token_id
model.config.pad_token_id = model.decoder.config.pad_token_id
model.config.eos_token_id = model.decoder.config.eos_token_id
model.config.max_length = 128
model.config.encoder.layerdrop = 0.0
model.config.use_cache = False
model.config.processor_class = "Wav2Vec2Processor"

# Load model without pretrained weights
config_encoder = AutoConfig.from_pretrained(encoder_id)
config_encoder.add_adapter = True
config_decoder = AutoConfig.from_pretrained(decoder_id)
config_decoder.is_decoder = True
config_decoder.add_cross_attention = True

config = SpeechEncoderDecoderConfig.from_encoder_decoder_configs(
    config_encoder, config_decoder
)
model_no_weights = SpeechEncoderDecoderModel(config=config)

model_no_weights.config.encoder.feat_proj_dropout = 0.0
model_no_weights.config.encoder.mask_time_prob = 0.0
model_no_weights.config.decoder_start_token_id = (
    model_no_weights.decoder.config.bos_token_id
)
model_no_weights.config.pad_token_id = model_no_weights.decoder.config.pad_token_id
model_no_weights.config.eos_token_id = model_no_weights.decoder.config.eos_token_id
model_no_weights.config.max_length = 128
model_no_weights.config.encoder.layerdrop = 0.0
model_no_weights.config.use_cache = False
model_no_weights.config.processor_class = "Wav2Vec2Processor"

# # Accessing the model configuration
# config_encoder = model_no_weights.config.encoder
# config_encoder.add_adapter = True
# config_decoder = model_no_weights.config.decoder
# # set decoder config to causal lm
# config_decoder.is_decoder = True
# config_decoder.add_cross_attention = True


# input_values = feature_extractor(
#     voxpopuli_head[0]["audio"]["array"], return_tensors="pt", sampling_rate=SAMPLING_RATE
# ).input_values

# generated_ids = model.generate(input_values, decoder_start_token_id=tokenizer.cls_token_id, )
# generated_text = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
# print(generated_text)

# # load its corresponding transcription and tokenize to generate labels
# labels = tokenizer(voxpopuli_head[0]["text"], return_tensors="pt").input_ids

In [4]:
from transformers import SpeechEncoderDecoderModel, AutoFeatureExtractor, AutoTokenizer
import torch


encoder_id = "facebook/wav2vec2-base"
decoder_id = "facebook/bart-base"
SAVE_PATH = "../../seq2seq_wav2vec2_bart-base"

model = SpeechEncoderDecoderModel.from_encoder_decoder_pretrained(
    encoder_id, decoder_id, encoder_add_adapter=True
)
model.config.encoder.feat_proj_dropout = 0.0
model.config.encoder.mask_time_prob = 0.0
model.config.decoder_start_token_id = model.decoder.config.bos_token_id
model.config.pad_token_id = model.decoder.config.pad_token_id
model.config.eos_token_id = model.decoder.config.eos_token_id
model.config.max_length = 128
model.config.encoder.layerdrop = 0.0
model.config.use_cache = False
model.config.processor_class = "Wav2Vec2Processor"

# check if generation works
# _ = model.generate(torch.ones((1, 2000)))

model.save_pretrained(SAVE_PATH)

feature_etxractor = AutoFeatureExtractor.from_pretrained(encoder_id)
feature_etxractor.save_pretrained(SAVE_PATH)
tokenizer = AutoTokenizer.from_pretrained(decoder_id)
tokenizer.save_pretrained(SAVE_PATH)

/home/matej/fitvut/dp_mit/.venv/lib/python3.11/site-packages/transformers/configuration_utils.py:311: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/wav2vec2-base and are newly initialized: ['wav2vec2.adapter.layers.0.conv.bias', 'wav2vec2.adapter.layers.0.conv.weight', 'wav2vec2.adapter.layers.1.conv.bias', 'wav2vec2.adapter.layers.1.conv.weight', 'wav2vec2.adapter.layers.2.conv.bias', 'wav2vec2.adapter.layers.2.conv.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BartForCausalLM were not initialized from the model checkpoint at facebook/bart-base and are newly

('../../seq2seq_wav2vec2_bart-base/tokenizer_config.json',
 '../../seq2seq_wav2vec2_bart-base/special_tokens_map.json',
 '../../seq2seq_wav2vec2_bart-base/vocab.json',
 '../../seq2seq_wav2vec2_bart-base/merges.txt',
 '../../seq2seq_wav2vec2_bart-base/added_tokens.json',
 '../../seq2seq_wav2vec2_bart-base/tokenizer.json')

In [2]:
# Load trained model
def display_weight_stats(model):
    for name, param in model.named_parameters():
        if param.requires_grad:
            mean = param.data.mean().item()
            std = param.data.std().item()
            print(f"Layer: {name}\t| Mean: {mean:.4f}\t| Std: {std:.4f}")
        else:
            print(f"Layer: {name}\t| No gradients")


def count_parameters(model):
    encoder_params = (
        sum(p.numel() for p in model.encoder.parameters() if p.requires_grad)
        if hasattr(model, "encoder")
        else 0
    )
    decoder_params = (
        sum(p.numel() for p in model.decoder.parameters() if p.requires_grad)
        if hasattr(model, "decoder")
        else 0
    )
    total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    return encoder_params, decoder_params, total_params

In [1]:
from transformers import AutoModelForSpeechSeq2Seq

model = AutoModelForSpeechSeq2Seq.from_pretrained("../seq2seq_wav2vec2_bart-base")

/home/matej/fitvut/dp_mit/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of the model checkpoint at ../seq2seq_wav2vec2_bart-base were not used when initializing SpeechEncoderDecoderModel: ['encoder.masked_spec_embed']
- This IS expected if you are initializing SpeechEncoderDecoderModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing SpeechEncoderDecoderModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [3]:
enc, dec, total = count_parameters(model)
print(f"Encoder: {enc}\t| Decoder: {dec}\t| Total: {total}")
model

Encoder: 104992384	| Decoder: 96103680	| Total: 201096064


SpeechEncoderDecoderModel(
  (encoder): Wav2Vec2Model(
    (feature_extractor): Wav2Vec2FeatureEncoder(
      (conv_layers): ModuleList(
        (0): Wav2Vec2GroupNormConvLayer(
          (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,), bias=False)
          (activation): GELUActivation()
          (layer_norm): GroupNorm(512, 512, eps=1e-05, affine=True)
        )
        (1-4): 4 x Wav2Vec2NoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
          (activation): GELUActivation()
        )
        (5-6): 2 x Wav2Vec2NoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(2,), stride=(2,), bias=False)
          (activation): GELUActivation()
        )
      )
    )
    (feature_projection): Wav2Vec2FeatureProjection(
      (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (projection): Linear(in_features=512, out_features=768, bias=True)
      (dropout): Dropout(p=0.0, inplace=False)
    )
   

In [8]:
display_weight_stats(model)

Layer: encoder.masked_spec_embed	| Mean: 0.1925	| Std: 0.2534
Layer: encoder.feature_extractor.conv_layers.0.conv.weight	| Mean: -0.0005	| Std: 0.1797
Layer: encoder.feature_extractor.conv_layers.0.layer_norm.weight	| Mean: 0.4893	| Std: 0.5753
Layer: encoder.feature_extractor.conv_layers.0.layer_norm.bias	| Mean: -0.0004	| Std: 0.0030
Layer: encoder.feature_extractor.conv_layers.1.conv.weight	| Mean: -0.0138	| Std: 0.0953
Layer: encoder.feature_extractor.conv_layers.2.conv.weight	| Mean: -0.0020	| Std: 0.0872
Layer: encoder.feature_extractor.conv_layers.3.conv.weight	| Mean: -0.0006	| Std: 0.0714
Layer: encoder.feature_extractor.conv_layers.4.conv.weight	| Mean: 0.0001	| Std: 0.0626
Layer: encoder.feature_extractor.conv_layers.5.conv.weight	| Mean: 0.0009	| Std: 0.0413
Layer: encoder.feature_extractor.conv_layers.6.conv.weight	| Mean: 0.0000	| Std: 0.0207
Layer: encoder.feature_projection.layer_norm.weight	| Mean: 0.3693	| Std: 0.0633
Layer: encoder.feature_projection.layer_norm.bias	

In [ ]:
# FOR DEBUGGING TO INSPECT THE AUDIO
import matplotlib.pyplot as plt
import librosa.display

# Load and display the original audio
audio_orig, sr_orig = librosa.load("../tmp/debug/2277-149896-0000_orig.mp3")
plt.figure(figsize=(14, 5))
librosa.display.waveshow(audio_orig, sr=sr_orig)
plt.title("Original Audio")
plt.show()

# Load and display the post-feature extracted audio
audio_post, sr_post = librosa.load("../tmp/debug/2277-149896-0000.mp3")
plt.figure(figsize=(14, 5))
librosa.display.waveshow(audio_post, sr=sr_post)
plt.title("Post-Feature Extracted Audio")
plt.show()